## Introduction to Parameter Sweep Applications


(Reference: *Writing Parallel Parameter Sweep Applications with pMatlab*, Hahn Kim, Albert Reuther and Jeremy Kepner, MIT Lincoln Laboratory, Lexinton, MA 02144
https://www.mit.edu/~kepner/pMatlab/pMatlabv2_param_sweep.pdf)

Parameter sweep applications are a class of application in which the same code is run multiple
times using unique sets of input parameter values. This includes varying one parameter over a
range of values or varying multiple parameters over a large multidimensional space. Examples
of parameter sweep applications are Monte Carlo simulations or parameter space searches.
In parameter sweep applications, each individual run is independent of all other runs. This
property is important for parallelizing parameter sweep applications because it means we can
formulate this type of problem in a leader-worker paradigm.  Each job is independent, it does not matter if the 415th job
completes after the 420th job completes. It is only important that each job completes and its
results are recorded. This type of application, in which individual processors do not
communicate with each other during processing, is known as pleasantly parallel or loosely coupled. 


A typical parameter sweep application consists of a for loop which repeatedly executes the same
code, usually in a function. A unique set of arguments is supplied to the function in each
iteration. Since the loop iterator has a unique value for each iteration, it can be used to compute
the set of arguments. A more advanced parameter sweep application may nest multiple for loops
and use multiple iterators to compute the input arguments. 


## Serial Octave Implementation

Let's look at a serial implementation of a parameter sweep workflow. In this Octave example, we use a function, sample_function.m, to fill a 2-D matrix that is M rows by N columns.  

Our 1st step is to add the path to the sample_function to our MATLAB path. The path below should point to the location of sample_function.m one directory up.

In [1]:
% basic parameter sweep code (serial)

addpath('../')


Next, we set the data sizes
* m = 3
* n = 16

and preallocate z, the matrix that will hold the results.  The pre-allocation is an optimization step. Note that we do not explictly provide the numeric values when we create z, but instead use the size variables defined above.


Finally, once all of the rows of the matrix has been filled we display the result.


In [2]:
% Set data sizes.
m = 3; % number of output arguments
n = 16; % number of independent iterations

% Create z, data output matrix.
z = zeros(n, m);

Now we have defined the number of rows, number of columns and the results matrix.  We are ready to start our calculations.  

In this example we want each row of the results matrix to hold
* the row number in column 1
* the number zero in column 2
* a value equal to 2.5 \* row number in column 3

In [3]:

% Loop over the local indices
for ii = 1:size(z, 1)
   % Calculate another argument based on the loop index (row number)
   my_other_arg = 2.5 * ii;

   % call a function with the index and other arguments, and 
   % store the result in a row
   z(ii, :) = sample_function(ii, 0, my_other_arg);
end % for ii



Finally, once all of the values have been computed and stored in the matrix, we display the result.

In [4]:
% Finalize the program
disp('SUCCESS');

% Finally, display the resulting matrix on the leader
disp(z);

SUCCESS
    1.00000    0.00000    2.50000
    2.00000    0.00000    5.00000
    3.00000    0.00000    7.50000
    4.00000    0.00000   10.00000
    5.00000    0.00000   12.50000
    6.00000    0.00000   15.00000
    7.00000    0.00000   17.50000
    8.00000    0.00000   20.00000
    9.00000    0.00000   22.50000
   10.00000    0.00000   25.00000
   11.00000    0.00000   27.50000
   12.00000    0.00000   30.00000
   13.00000    0.00000   32.50000
   14.00000    0.00000   35.00000
   15.00000    0.00000   37.50000
   16.00000    0.00000   40.00000


Once the results are complete, save your data to a file.  To save it to a mat file, we use the save command.

In [5]:
save('Matrix_result.mat','z')

To save the data to a text file for use in another Octave script or even a script in another language, we recommend csvwrite.  For this data the command is 

In [6]:
csvwrite('paramSweepMat.txt',z)